In [72]:
# a) Report descriptive statistics (mean, min, max, and standard deviation) for each of
# the untransformed variables in the data set.

import pandas as pd
import numpy as np

eats = pd.read_csv('C:\\Users\\dante\\Downloads\\HW3.csv', delimiter=',')
dummy = pd.get_dummies(eats['QRT'])
df = pd.concat([eats,dummy], axis=1)
df['M'] = (df['Q1']*df['P1']) + (df['Q2']*df['P2']) + (df['Q3']*df['P3'])
df = df.rename(columns = {"1": "QRT1", "2":"QRT2", "3": "QRT3", "4":"QRT4"})
df.columns = ['YR', 'QRT', 'Q1', 'Q2', 'Q3', 'P1', 'P2', 'P3', 'QRT1', 'QRT2', 'QRT3', 'QRT4', 'M']

df['lP1'] = np.log(df['P1'])
df['lP2'] = np.log(df['P2'])
df['lP3'] = np.log(df['P3'])
df['lM'] = np.log(df['M'])
df.columns = ['YR', 'QRT', 'Q1', 'Q2', 'Q3', 'P1', 'P2', 'P3', 'QRT1', 'QRT2', 'QRT3', 'QRT4', 'M', 'lP1', 'lP2', 'lP3', 'lM']
print(f'df.head(): \n{df.head()}\n')

Data_summary = df.describe()
print(f'Data_summary:\n{Data_summary}\n')

df.head(): 
     YR  QRT       Q1       Q2       Q3        P1     P2     P3  QRT1  QRT2  \
0  1982    2  18.7968  12.2607  12.8340  242.9667  179.2  72.13     0     1   
1  1982    3  19.9882  11.5797  12.9951  244.0333  195.7  72.33     0     0   
2  1982    4  19.4242  12.5368  11.8858  233.1333  198.0  69.20     0     0   
3  1983    1  19.2252  12.1140  12.4026  233.8667  193.6  69.53     1     0   
4  1983    2  19.3361  12.7472  13.1290  240.9333  181.0  68.73     0     1   

   QRT3  QRT4            M       lP1       lP2       lP3        lM  
0     0     0  7689.830327  5.492924  5.188503  4.278470  8.947654  
1     1     0  8083.869280  5.497305  5.276583  4.281239  8.997626  
2     0     1  7833.211606  5.451610  5.288267  4.237001  8.966128  
3     0     0  7703.757259  5.454751  5.265794  4.241758  8.949463  
4     0     0  7868.309752  5.484520  5.198497  4.230186  8.970599  

Data_summary:
                YR        QRT         Q1         Q2         Q3          P1  \
count 

In [70]:
# b) What is the ML estimator for the parameters of the consumer demand model?

import statsmodels.formula.api as smf
import patsy as pt

y, X = pt.dmatrices('Q1 ~ lP1 + lP2 + lP3 + lM + QRT1 + QRT2 + QRT3', data=df, return_type='dataframe')
print(f'X.head(): \n{X.head()}\n')

X = np.array(X)
y = np.array(y).reshape(n,1)
b = np.linalg.inv(X.T @ X) @ X.T @ y
print(f'b: \n{b}\n')

u_hat = y - X @ b
sigsq_hat = (u_hat.T @ u_hat) / n
SER = np.sqrt(sigsq_hat)
print(f'SER: \n{SER}\n')

Vbeta_hat = sigsq_hat * np.linalg.inv(X.T @ X)
se = np.sqrt(np.diagonal(Vbeta_hat))
print(f'se: {se}\n')

div_vec = np.divide(b , se)
print(f' \n{div_vec}\n')

tstats = np.diagonal(div_vec)
print(f'tstats: \n{tstats}\n')

df.head(): 
     YR  QRT       Q1       Q2       Q3        P1     P2     P3  QRT1  QRT2  \
0  1982    2  18.7968  12.2607  12.8340  242.9667  179.2  72.13     0     1   
1  1982    3  19.9882  11.5797  12.9951  244.0333  195.7  72.33     0     0   
2  1982    4  19.4242  12.5368  11.8858  233.1333  198.0  69.20     0     0   
3  1983    1  19.2252  12.1140  12.4026  233.8667  193.6  69.53     1     0   
4  1983    2  19.3361  12.7472  13.1290  240.9333  181.0  68.73     0     1   

   QRT3  QRT4            M       lP1       lP2       lP3        lM  
0     0     0  7689.830327  5.492924  5.188503  4.278470  8.947654  
1     1     0  8083.869280  5.497305  5.276583  4.281239  8.997626  
2     0     1  7833.211606  5.451610  5.288267  4.237001  8.966128  
3     0     0  7703.757259  5.454751  5.265794  4.241758  8.949463  
4     0     0  7868.309752  5.484520  5.198497  4.230186  8.970599  

X.head(): 
   Intercept       lP1       lP2       lP3        lM  QRT1  QRT2  QRT3
0        1.0  5.

In [33]:
# c) Report your ML estimates and t-values for each parameter. Explain. Interpret
# both economically and statistically your regression results.

# All values shown in results above.

# The negative coefficients on the prices of all 3 goods reflect that the goods
# are compliments. If the price increases on one of them, demand for all 3 drops.
# The positive coefficient on M also reflects this, since consumption of all 3
# goods rises together.

# The high coefficient on M should be considered carefully since the value Q1 we
# are regressing towards is included in the formula to generate M.

# The varying values of QRT1-QRT3 reflect a seasonality in the demand for this good,
# but they are all still positive so the good has utility at least January-September.

# All variables except QRT1 are statistically significant according to the T stats.

In [77]:
# d) Is your ML of the regression parameters in part c) 
# unbiased, consistent, and asymptotically normally distributed? Explain

reg = smf.ols(formula='Q1 ~ lP1 + lP2 + lP3 + lM + QRT1 + QRT2 + QRT3', data=df)
results = reg.fit()
print(f'results.summary(): \n{results.summary()}\n')


# The parameter estimates are unbiased, consistent, and asymptotically normally
# distributed because they are the same as an OLS, meaning the classical 
# assumptions have been met.

results.summary(): 
                            OLS Regression Results                            
Dep. Variable:                     Q1   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.951
Method:                 Least Squares   F-statistic:                     178.5
Date:                Thu, 03 Mar 2022   Prob (F-statistic):           2.32e-36
Time:                        15:26:09   Log-Likelihood:                -11.535
No. Observations:                  65   AIC:                             39.07
Df Residuals:                      57   BIC:                             56.46
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     43.5775     12.889

In [75]:
# e) Specify and test the null hypothesis that prices have not impact on 
# quantity demanded.

hypotheses = ['np.log(P1) = 0', 'np.log(P2) = 0', 'np.log(P3) = 0']
ftest = results.f_test(hypotheses)
fpval = ftest.pvalue
print(f'fpval: {fpval}\n')

# P value is very small, we can reject null hypothesis

fpval: 2.5575440300805035e-17



In [76]:
# f) Report the ML estimate for the variance sigma^2. 
# Is it unbiased or consistent? Explain

print(f'sigsq_hat: \n{sigsq_hat}\n')

# Under MLE, sigma squared is biased
# It is consistent because as n grows, sigma squared approaches its true value.

sigsq_hat: 
[[0.0834956]]



In [44]:
# g) Report (and interpret) the adjusted R-square value.

# Adjusted R-square = 0.951
# 95.1% of the variation in Q1 is explained by the variables in the model.

In [79]:
# h) Report your ML estimates of the price elasticities at the mean. Explain. Interpret
# both economically and statistically your results. 

lp1_elast_at_mean = (5.567377/17.825522)*(-16.3312) 
print(lp1_elast_at_mean)

lp2_elast_at_mean = (5.314968/17.825522)*(-2.5818) 
print(lp2_elast_at_mean)

lp3_elast_at_mean = (4.456142/17.825522)*(-2.9635) 
print(lp3_elast_at_mean)

# The mean of the log of price / the mean quantity of good 1 * the 
# coefficient for that log of price

# The log of the prices of goods 2 and 3 are inelastic to the quantity of
# good one. The price of good 1 is elastic.

-5.100661134209702
-0.7698054723109933
-0.740835349281777
